In [2]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [ ]:
def smoothed_mean(X, y, kf, alpha, global_mean=None, columns=None):

    if global_mean is None:
        global_mean = np.mean(y)
        
    if columns is None:
        columns = X.columns
    
    unic_values_X = {}
    for column in columns:
        X[column + '_agr'] = np.NaN
        unic_values_X[column] = set(X[column])
            
    
    for train_indexes, fill_indexes in kf.split(X, y):
        for column in columns:
            for value in unic_values_X[column]:
                slice_train = X.ix[train_indexes][column] == value
                n_rows = slice_train.sum()
                mean_y = 0
                if n_rows > 0:
                    mean_y = y[slice_train[slice_train].index].mean()
                smoothed_mean = (mean_y * n_rows + global_mean * alpha) / (n_rows + alpha)
                
                slice_fill = X.ix[fill_indexes][column] == value
                slice_fill_indexes = slice_fill[slice_fill].index
                X[column + '_agr'][slice_fill_indexes] = smoothed_mean
    return X

In [3]:
train_X = pd.DataFrame(np.random.randint(low=10, size=(10,5)), columns=['a', 'b', 'c', 'd', 'e'])
train_y = np.random.binomial(1, 0.7, 10)

test_X = pd.DataFrame(np.random.randint(low=10, size=(10,5)), columns=['a', 'b', 'c', 'd', 'e'])
test_y = np.random.binomial(1, 0.7, 10)

In [4]:
alpha = 10
global_mean = train_y.mean()
columns = ['b', 'd']

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
kf_fill = KFold(n_splits=3, shuffle=True, random_state=42)
for train_index, test_index in kf.split(train_X, train_y):
    
    smoothed_mean(X=train_X.ix[train_index], y=train_y[train_index], alpha=alpha, global_mean=global_mean, columns=columns)
    
    """
    print('train_X \n', train_X.ix[train_index])
    print('------')
    print('train_y \n', train_y[train_index])
    print('------')
    print('test_X \n', train_X.ix[test_index])
    print('------')
    print('test_y \n', train_y[test_index])
    print('---------------------------------------------------')
    """
    